In [119]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime

try:

    df_list = []

    for i in range(1, 31):
        try:
            filename = f'resources/transactions_summary ({i}).csv'
            df = pd.read_csv(filename)
            df_list.append(df)
            print(f'Successfully loaded file {filename}')
            # Perform any data analysis or manipulation you need
        except FileNotFoundError:
            pass

    shake_data_df = pd.concat(df_list, ignore_index=True)
    shake_data_df['Date'] = pd.to_datetime(shake_data_df['Date']).dt.strftime('%Y-%m-%d')

except FileNotFoundError:
    print("File not found. Please check the filename and path and try again.")





Successfully loaded file resources/transactions_summary (12).csv
Successfully loaded file resources/transactions_summary (13).csv


In [120]:
shake_data_df1 = shake_data_df.drop(['Debit Currency','Direction', 'Source / Destination', 'Blockchain Transaction ID'], axis=1)
mask = shake_data_df1['Transaction Type'].isin(['purchase/sale'])
new_shake_data_df = shake_data_df[mask]
mask2 = shake_data_df['Credit Currency'].isin(['BTC'])
new_shake_data_df = shake_data_df[mask2]
new_shake_data_df

,Transaction Type,Date,Amount Debited,Debit Currency,Amount Credited,Credit Currency,Buy / Sell Rate,Direction,Spot Rate,Source / Destination,Blockchain Transaction ID
1,purchase/sale,2022-02-22,100.0,CAD,2.045910e-03,BTC,48877.8439,purchase,NaN,NaN,NaN
3,shakingsats,2022-02-22,NaN,NaN,1.000000e-06,BTC,NaN,credit,48461.8017,NaN,NaN
5,purchase/sale,2022-02-22,30.0,CAD,6.108500e-04,BTC,49111.8218,purchase,NaN,NaN,NaN
7,shakingsats,2022-02-23,NaN,NaN,1.100000e-06,BTC,NaN,credit,49023.2470,NaN,NaN
13,card cashbacks,2022-02-24,NaN,NaN,4.431000e-05,BTC,NaN,credit,45254.2536,@cashbacks,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6276,shakesquads,2023-07-24,NaN,NaN,1.680000e-06,BTC,NaN,credit,38267.2206,NaN,NaN
6277,shakingsats,2023-07-25,NaN,NaN,6.000000e-07,BTC,NaN,credit,38341.5100,NaN,NaN
6278,card cashbacks,2023-07-25,NaN,NaN,1.555000e-05,BTC,NaN,credit,38348.5462,@cashbacks,NaN
6279,purchase/sale,2023-07-25,6.0,CAD,1.532500e-04,BTC,39150.5316,purchase,NaN,NaN,NaN


In [121]:
buy_sell_rate = new_shake_data_df['Buy / Sell Rate'].mean()
stack = new_shake_data_df['Amount Credited'].sum()
cash = new_shake_data_df['Amount Debited'].sum()
print(buy_sell_rate)
print(stack)
print(cash)


36290.18973958338
0.65092243
25361.870000000003


In [122]:
new_shake_data_df['weightedbuy'] = new_shake_data_df['Buy / Sell Rate'] * new_shake_data_df['Amount Debited']
weighted_average=new_shake_data_df['weightedbuy'].sum() / new_shake_data_df['Buy / Sell Rate'].mean()
print(weighted_average)

35509.2545752275


C:\Users\16474\AppData\Local\Temp\ipykernel_12112\1686005046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_shake_data_df['weightedbuy'] = new_shake_data_df['Buy / Sell Rate'] * new_shake_data_df['Amount Debited']
